# Ictonyx Example: PyTorch Regression Variability Study

This notebook trains a small network on a synthetic regression problem and reports
the distribution of validation MSE across runs.

**Requirements:** `pip install ictonyx torch`

In [1]:
import numpy as np
import torch
import torch.nn as nn

import ictonyx as ix
from ictonyx import (
    ModelConfig,
    PyTorchModelWrapper,
    ArraysDataHandler,
    run_variability_study,
)

print(f"Ictonyx v{ix.__version__}")
print(f"PyTorch v{torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

2026-02-16 13:47:50.000198: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-16 13:47:50.025657: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-16 13:47:50.025684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-16 13:47:50.026490: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-16 13:47:50.030756: I tensorflow/core/platform/cpu_feature_guar

Ictonyx v0.3.2
PyTorch v2.10.0+cu128
Device: cuda


## 1. Generate Synthetic Data

We create a simple linear relationship with 5 features and Gaussian noise.
The true weights are known, so we can verify the model is learning something real.

In [2]:
rng = np.random.RandomState(0)
X = rng.randn(500, 5).astype(np.float32)
true_weights = np.array([1.5, -2.0, 0.5, 0.0, 3.0], dtype=np.float32)
y = X @ true_weights + rng.randn(500).astype(np.float32) * 0.3

data_handler = ArraysDataHandler(X, y)

print(f"Samples: {len(X)}")
print(f"Features: {X.shape[1]}")
print(f"True weights: {true_weights}")
print(f"Noise level: 0.3")

Samples: 500
Features: 5
True weights: [ 1.5 -2.   0.5  0.   3. ]
Noise level: 0.3


## 2. Define the Model Builder

For regression, set `task='regression'` and use an appropriate loss (MSELoss).
The wrapper will track MSE instead of accuracy during training.

In [3]:
def create_regressor(config: ModelConfig) -> PyTorchModelWrapper:
    model = nn.Sequential(
        nn.Linear(5, 32),
        nn.ReLU(),
        nn.Linear(32, 16),
        nn.ReLU(),
        nn.Linear(16, 1),
    )
    return PyTorchModelWrapper(
        model,
        criterion=nn.MSELoss(),
        optimizer_class=torch.optim.Adam,
        optimizer_params={'lr': config.get('learning_rate', 0.005)},
        task='regression',
    )

print(repr(create_regressor(ModelConfig())))

2026-02-16 13:47:58.097074: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


Ictonyx BaseModelWrapper(id='', type='Sequential', is_trained=No)


2026-02-16 13:47:58.098329: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-16 13:47:58.099120: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-16 13:47:58.100888: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

## 3. Run the Variability Study

10 runs, 50 epochs each. The question: how much does final MSE vary
across different random initializations?

In [4]:
config = ModelConfig({
    'epochs': 50,
    'batch_size': 32,
    'learning_rate': 0.005,
    'verbose': 0,
})

results = run_variability_study(
    model_builder=create_regressor,
    data_handler=data_handler,
    model_config=config,
    num_runs=10,
    seed=42,
)

Loading and preparing data...
Array splits - Train: 350, Val: 50, Test: 100
Data loaded successfully

Starting Variability Study
  Runs: 10
  Epochs per run: 50
  Execution mode: in standard mode
  Seed: 42



/home/appuser/projects/ictonyx/ictonyx/memory.py:197: UserWarning: TensorFlow setup failed: Physical devices cannot be modified after being initialized
  warnings.warn(f"TensorFlow setup failed: {e}")
Variability Study: 100%|███████| 10/10 [00:29<00:00,  2.98s/run, val_mse=0.0924]


--------------------------------------------------

Study Summary:
  Successful runs: 10/10
  train_loss: 0.0681 (SD = 0.0035)
  val_loss: 0.1118 (SD = 0.0171)
  val_mse: 0.1118 (SD = 0.0171)


## 4. Examine Results

In [5]:
print(results.summarize())

Variability Study Results
Successful runs: 10
Seed: 42
train_loss:
  Mean: 0.0681
  Std:  0.0035
  Min:  0.0620
  Max:  0.0752
val_loss:
  Mean: 0.1118
  Std:  0.0171
  Min:  0.0924
  Max:  0.1549
val_mse:
  Mean: 0.1118
  Std:  0.0171
  Min:  0.0924
  Max:  0.1549


In [6]:
print("Available metrics:", results.get_available_metrics())
print()
val_mse = results.get_metric_values('val_mse')
print("Per-run val_mse:")
for i, mse in enumerate(val_mse, 1):
    print(f"  Run {i}: {mse:.4f}")

print(f"\nBest:  {min(val_mse):.4f}")
print(f"Worst: {max(val_mse):.4f}")
print(f"Range: {max(val_mse) - min(val_mse):.4f}")

Available metrics: ['train_loss', 'val_loss', 'val_mse']

Per-run val_mse:
  Run 1: 0.1249
  Run 2: 0.1013
  Run 3: 0.1101
  Run 4: 0.1005
  Run 5: 0.1549
  Run 6: 0.1033
  Run 7: 0.1218
  Run 8: 0.1048
  Run 9: 0.1044
  Run 10: 0.0924

Best:  0.0924
Worst: 0.1549
Range: 0.0625


In [7]:
# Summary DataFrame
results.to_dataframe()

,run_id,final_train_loss,final_val_loss,final_val_mse,test_loss,test_mse
0,1,0.069311,0.124906,0.124906,0.121437,0.121437
1,2,0.067299,0.101263,0.101263,0.101542,0.101542
2,3,0.075225,0.110128,0.110128,0.106627,0.106627
3,4,0.068154,0.100534,0.100534,0.101477,0.101477
4,5,0.071668,0.154897,0.154897,0.121365,0.121365
5,6,0.067011,0.103286,0.103286,0.097865,0.097865
6,7,0.063861,0.121755,0.121755,0.102695,0.102695
7,8,0.069882,0.104754,0.104754,0.102358,0.102358
8,9,0.066952,0.104363,0.104363,0.099667,0.099667
9,10,0.062038,0.092365,0.092365,0.095350,0.095350
